# Introdução ao Deep Learning com PyTorch

Neste caderno, você terá uma introdução ao [PyTorch](http://pytorch.org/), que é uma estrutura para construção e treinamento de redes neurais (NN). ``PyTorch`` se comporta de várias maneiras como os arrays que você conhece e adora do Numpy. Afinal, esses arrays Numpy são apenas *tensores*. PyTorch pega esses tensores e simplifica movê-los para GPUs para o processamento mais rápido necessário ao treinar redes neurais. Ele também fornece um módulo que calcula gradientes automaticamente (para retropropagação!) E outro módulo específico para construção de redes neurais. No geral, o PyTorch acaba sendo mais coerente com o **Python** e a pilha ``Numpy/Scipy`` em comparação com o *TensorFlow* e outros frameworks.



## Redes neurais

O Deep Learning é baseado em [redes neurais artificiais](https://en.wikipedia.org/wiki/Artificial_neural_network) que existem de alguma forma desde o final dos anos 1950. As redes são construídas a partir de partes individuais que se aproximam dos neurônios, normalmente chamadas de unidades ou simplesmente “neurônios”. Cada unidade possui um certo número de entradas ponderadas. Essas entradas ponderadas são somadas (uma combinação linear) e depois passadas por uma função de ativação para obter a saída da unidade.

<img src="assets/simple_neuron.png" largura=400px>






Matematicamente isso se parece com:

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Com vetores, este é o produto escalar/interno de dois vetores:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$


## Tensores

Acontece que os cálculos de redes neurais são apenas um monte de operações de álgebra linear em *tensores*, uma generalização de matrizes. Um vetor é um tensor unidimensional, uma matriz é um tensor bidimensional, uma matriz com três índices é um tensor tridimensional (imagens coloridas RGB, por exemplo). A estrutura de dados fundamental para redes neurais são tensores e PyTorch (assim como praticamente todas as outras estruturas de aprendizado profundo) é construído em torno de tensores.

<img src="assets/tensor_examples.svg" width=600px>

Com o básico abordado, é hora de explorar como podemos usar o PyTorch para construir uma rede neural simples

In [2]:
# First, import PyTorch.
import torch

In [124]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [125]:
### Generate some data.
torch.manual_seed(7) #Define a semente do gerador de números aleatórios para garantir reprodutibilidade nos resultados.

#Gera uma matriz de dimensão 1×5 contendo valores aleatórios de uma distribuição normal 
features = torch.randn((1, 5))
# Gera uma matriz de pesos aleatórios com as mesmas dimensões que as características. 
#Esta matriz será usada para ponderar os recursos na regressão linear.
weights = torch.randn_like(features)
# and a true bias term.
bias = torch.randn((1, 1))

Acima, gerei dados que podemos usar para obter a saída de nossa rede simples. Isso tudo é aleatório por enquanto, daqui para frente começaremos a usar dados normais. Percorrendo cada linha relevante:

`features = torch.randn((1, 5))` cria um tensor com forma `(1, 5)`, uma linha e cinco colunas, que contém valores distribuídos aleatoriamente de acordo com a distribuição normal com média zero e padrão desvio de um.

`weights = torch.randn_like(features)` cria outro tensor com a mesma forma de `features`, novamente contendo valores de uma distribuição normal.

Finalmente, `bias = torch.randn((1, 1))` cria um único valor de uma distribuição normal.

Os tensores PyTorch podem ser adicionados, multiplicados, subtraídos, etc., assim como os arrays Numpy. Em geral, você usará tensores PyTorch praticamente da mesma forma que usaria matrizes Numpy. Eles vêm com alguns benefícios interessantes, como aceleração de GPU, que veremos mais tarde. Por enquanto, use os dados gerados para calcular a saída desta rede simples de camada única.
> **Exercício**: Calcule a saída da rede com recursos de entrada `recursos`, pesos `pesos` e polarização `viés`. Semelhante ao Numpy, o PyTorch tem uma função [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), bem como um método `.sum()` em tensores, para obter somas. Use a função `ativação` definida acima como a função de ativação.

In [126]:
## Calculate the output of this network using the weights and bias tensors.

# Calcula a saída da rede neural
output = activation(torch.sum(features * weights) + bias)

print(output)

tensor([[0.1595]])


Você pode fazer a multiplicação e a soma na mesma operação usando uma multiplicação de matrizes. Em geral, você desejará usar multiplicações de matrizes, pois elas são mais eficientes e aceleradas usando bibliotecas modernas e computação de alto desempenho em GPUs.

Aqui, queremos fazer uma multiplicação matricial dos features and the weights. Para isso podemos usar [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) ou [`torch.matmul()`](https:// pytorch.org/docs/stable/torch.html#torch.matmul), que é um pouco mais complicado e suporta transmissão. Se tentarmos fazer isso com `features` e `weights` como estão, obteremos um erro

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

Ao construir redes neurais em qualquer estrutura, você verá isso com frequência. Muitas vezes. O que está acontecendo aqui é que nossos tensores não têm a forma correta para realizar uma multiplicação de matrizes. Lembre-se que para multiplicações de matrizes, o número de colunas no primeiro tensor deve ser igual ao número de linhas no segundo tensor. Ambos `features` e `weight` têm a mesma forma, `(1, 5)`. Isso significa que precisamos mudar a forma dos `weight` para que a multiplicação da matriz funcione.

**Nota:** Para ver a forma de um tensor chamado `tensor`, use `tensor.shape`. Se você estiver construindo redes neurais, usará esse método com frequência.

Existem algumas opções aqui: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`]( https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch .Tensor.view) e [`torch.transpose(weights,0,1)`](https://pytorch.org/docs/master/generated/torch.transpose.html).

* `weights.reshape(a, b)` retornará um novo tensor com os mesmos dados que `weights` com tamanho `(a, b)` às vezes, e às vezes um clone, já que copia os dados para outra parte do memória.
* `weights.resize_(a, b)` retorna o mesmo tensor com uma forma diferente. Porém, se a nova forma resultar em menos elementos que o tensor original, alguns elementos serão removidos do tensor (mas não da memória). Se a nova forma resultar em mais elementos do que o tensor original, os novos elementos não serão inicializados na memória. Aqui devo observar que o sublinhado no final do método indica que este método é executado **no local**. Aqui está um ótimo tópico no fórum para [ler mais sobre operações no local](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) no PyTorch.
* `weights.view(a, b)` retornará um novo tensor com os mesmos dados que `weights` com tamanho `(a, b)`.
* `torch.transpose(weights,0,1)` retornará o tensor de pesos transpostos. Isso retorna a versão transposta do tensor inpjut ao longo de dim 0 e dim 1. Isso é eficiente, pois não especificamos as dimensões reais dos pesos.

Eu costumo usar `.view()`, mas qualquer um dos três métodos funcionará para isso. Então, agora podemos remodelar `weights` para ter cinco linhas e uma coluna com algo como `weights.view(5, 1)`.

Mais uma abordagem é usar `.t()` para transpor o vetor de pesos, no nosso caso da forma (1,5) para (5,1).
> **Exercício**: Calcule a saída de nossa pequena rede usando multiplicação de matrizes.

In [127]:
## Calculate the output of this network using matrix multiplication
weights = weights.view(5, 1)

output = activation(torch.mm(features, weights) + bias)

print(output)

tensor([[0.1595]])


### Empilhe-os!

É assim que você pode calcular a saída de um único neurônio. O verdadeiro poder desse algoritmo acontece quando você começa a empilhar essas unidades individuais em camadas e pilhas de camadas, em uma rede de neurônios. A saída de uma camada de neurônios torna-se a entrada da próxima camada. Com múltiplas unidades de entrada e unidades de saída, agora precisamos expressar os pesos como uma matriz.

<img src='assets/multilayer_diagram_weights.png' largura=450px>

A primeira camada mostrada abaixo são as entradas, compreensivelmente chamadas de **camada de entrada**. A camada intermediária é chamada de **camada oculta**, e a camada final (à direita) é a **camada de saída**. Podemos expressar esta rede matematicamente com matrizes novamente e usar a multiplicação de matrizes para obter combinações lineares para cada unidade em uma operação. Por exemplo, a camada oculta ($h_1$ e $h_2$ aqui) pode ser calculada

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

A saída para esta pequena rede é encontrada tratando a camada oculta como entradas para a unidade de saída. A saída da rede é expressa simplesmente

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [128]:
### Generate some data.

torch.manual_seed(7)  # Define a semente aleatória para garantir a previsibilidade.

# Os recursos são 3 variáveis normais aleatórias.
features = torch.randn((1, 3))

# Define o tamanho de cada camada em nossa rede.
n_input = features.shape[1]     # Número de unidades de entrada, deve coincidir com o número de recursos de entrada.
n_hidden = 2                    # Número de unidades ocultas.
n_output = 1                    # Número de unidades de saída.

# Pesos para as entradas para a camada oculta.
W1 = torch.randn(n_input, n_hidden)
# Pesos para a camada oculta para a camada de saída.
W2 = torch.randn(n_hidden, n_output)

# and bias terms para as camadas oculta e de saída
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

**Exercício:** Calcule a saída para esta rede multicamadas usando os pesos `W1` e `W2` e as tendências, `B1` e `B2`.

In [129]:
## Your solution here.
# Calcula a saída da camada oculta
hidden_output = activation(torch.mm(features, W1) + B1)

# Calcula a saída da camada de saída
output = activation(torch.mm(hidden_output, W2) + B2)

print(output)

tensor([[0.3171]])


Se você fez isso corretamente, deverá ver a saída `tensor([[ 0.3171]])`.

O número de unidades ocultas é um parâmetro da rede, geralmente chamado de **hiperparâmetro** para diferenciá-lo dos parâmetros de pesos e bias. Como você verá mais tarde, quando discutirmos o treinamento de uma rede neural, quanto mais unidades ocultas uma rede tiver, e quanto mais camadas, mais capaz ela será de aprender com os dados e fazer previsões precisas.

## Numpy para torch e vice-versa

Seção de bônus especial! PyTorch tem um ótimo recurso para conversão entre arrays Numpy e tensores Torch. Para criar um tensor a partir de um array Numpy, use `torch.from_numpy()`. Para converter um tensor em um array Numpy, use o método `.numpy()`.

In [3]:
import numpy as np
np.set_printoptions(precision=8)
a = np.random.rand(4,3)
a

array([[0.77972024, 0.92298773, 0.77081473],
       [0.63043875, 0.39674868, 0.07300527],
       [0.22249923, 0.24156571, 0.79578098],
       [0.09626836, 0.45104908, 0.51498427]])

In [4]:
torch.set_printoptions(precision=8)
b = torch.from_numpy(a)
b

tensor([[0.77972024, 0.92298773, 0.77081473],
        [0.63043875, 0.39674868, 0.07300527],
        [0.22249923, 0.24156571, 0.79578098],
        [0.09626836, 0.45104908, 0.51498427]], dtype=torch.float64)

In [ ]:
b.numpy()

array([[0.61995741, 0.33155467, 0.8174536 ],
       [0.34563545, 0.15472967, 0.23112194],
       [0.57911665, 0.7242849 , 0.41691137],
       [0.68045155, 0.49653628, 0.32549209]])

A memória é compartilhada entre o array Numpy e o tensor Torch, portanto, se você alterar os valores no local de um objeto, o outro também mudará.

In [5]:
# Multiply PyTorch Tensor by 2, in place.
b.mul_(2)

tensor([[1.55944048, 1.84597547, 1.54162946],
        [1.26087751, 0.79349736, 0.14601054],
        [0.44499845, 0.48313142, 1.59156196],
        [0.19253673, 0.90209817, 1.02996854]], dtype=torch.float64)

In [6]:
# Numpy array matches new values from Tensor.
a


array([[1.55944048, 1.84597547, 1.54162946],
       [1.26087751, 0.79349736, 0.14601054],
       [0.44499845, 0.48313142, 1.59156196],
       [0.19253673, 0.90209817, 1.02996854]])